# Caso Advanced Analytics: 
## Predicción de Fugas de Clientes para una Institución Financiera
Autor: Rolf Traeger Brautigam
=========

# Inicio
## Import librarys

In [1]:
# ! conda install plotnine
# ! conda install pandas
# ! conda install scikit-learn
# ! conda install -c anaconda git


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from plotnine import (
ggplot,
geom_histogram,
aes,
geom_col,
theme,
element_text,
ggtitle,
scale_x_discrete,
facet_grid,
position_dodge,
)

## Funciones utiles

In [4]:
import exploratory_lib as exp_lib

In [5]:
dic_med=exp_lib.class_diccionarios()
dic_med.set_list_num(list(range(1,13)))
dic_med.set_list_name(exp_lib.array_mes_name)
dic_med.set_dictionary()

In [6]:
dic_med.int_to_name(1)

'Enero'

In [7]:
dic_dianom=exp_lib.class_diccionarios()
dic_dianom.set_list_num(list(range(1,8)))
dic_dianom.set_list_name(['Lunes', 'Martes', 'Miercoles','Jueves','Viernes', 'Sabado', 'Domingo'])
dic_dianom.set_dictionary()
dic_dianom.name_to_num("LuneS")

1

## Extract CSV y transform using owns librarys

In [8]:
list_name_file_1 = [
    'aportes_rescates_12m',      #.csv
    'cuenta_banco',      #.csv
    'demograficos',      #.csv
    'deudas',        #.csv
    'id_label',      #.csv
    ]
list_name_file_cierre_patrimonio=[
'cierre_patrimonio_202202',        #.csv
'cierre_patrimonio_202203',        #.csv
'cierre_patrimonio_202204',        #.csv
'cierre_patrimonio_202205',        #.csv
'cierre_patrimonio_202206',        #.csv
'cierre_patrimonio_202207',        #.csv
'cierre_patrimonio_202208',        #.csv
'cierre_patrimonio_202209',        #.csv
'cierre_patrimonio_202210',        #.csv
'cierre_patrimonio_202211',        #.csv
'cierre_patrimonio_202212',        #.csv
'cierre_patrimonio_202301',        #.csv
'cierre_patrimonio_202302',        #.csv
]

In [9]:
aportes_rescates_12m = pd.read_csv("bases/aportes_rescates_12m.csv",
                        encoding="utf-8",sep=",",nrows=0, 
                        index_col=0)
# exp_lib.extract_csv("bases","aportes_rescates_12m",',')
cuenta_banco = pd.read_csv("bases/cuenta_banco.csv",
                        encoding="utf-8",sep=",",nrows=0
                        ,index_col=None)
# exp_lib.extract_csv("bases","cuenta_banco",',')
demograficos =  pd.read_csv("bases/demograficos.csv",
                        encoding="utf-8",sep=",",nrows=0
                        ,index_col=None)
deudas =   pd.read_csv("bases/demograficos.csv",
                        encoding="utf-8",sep=",",nrows=0
                        ,index_col=None)
# exp_lib.extract_csv("bases","deudas",',')
id_label = pd.read_csv("bases/id_label.csv",
                        encoding="utf-8",sep=",",nrows=0
                        ,index_col=None)


In [10]:
aportes_rescates_12m = exp_lib.extract_csv("bases","aportes_rescates_12m",',',0)
cuenta_banco = exp_lib.extract_csv("bases","cuenta_banco",',')
demograficos = exp_lib.extract_csv("bases","demograficos",',')
deudas = exp_lib.extract_csv("bases","deudas",',')
id_label = exp_lib.extract_csv("bases","id_label",',')


Dimensiones: (37146, 3)
Columnas 
 n_rescates, n_aportes, id
Dimensiones: (71664, 2)
Columnas 
 id, clasificacion_cliente
Dimensiones: (36970, 10)
Columnas 
 id, fec_nacim, flg_empleado, gls_nacioncorta, gls_residencialarga, gls_profesionlarga, cod_sexo, gls_cod_estcivillarga, gls_niv_educaclarga, gls_sep_bieneslarga
Dimensiones: (367543, 16)
Columnas 
 id, periodo, entidades, disponible, deuda_1, deuda_2, deuda_3, deuda_4, deuda_5, mora_1, mora_2, mora_3, mora_4, mora_5, mora_6, mora_7
Dimensiones: (37146, 2)
Columnas 
 id, label


In [11]:
df_list_cierre_patrimonio = [
    exp_lib.extract_csv(
    "bases/cierre_patrimonio",
    i,
    ',',0).drop(columns=["Unnamed: 0.1"])
    for i in list_name_file_cierre_patrimonio
    ]
df_cierre_patrimonio = \
    pd.concat(df_list_cierre_patrimonio)
del df_list_cierre_patrimonio

Dimensiones: (30674, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (31111, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (31425, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (31997, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (32423, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (33010, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (33994, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (34762, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (35420, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (36341, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (37027, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (36444, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (35956, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fec

## set columns format

In [12]:
exp_lib.columns_cast(
    aportes_rescates_12m,
    ["n_rescates","n_aportes"],
    "float64")
exp_lib.columns_cast(
    aportes_rescates_12m,
    ["id"],
    "int64")
aportes_rescates_12m.head(2)

n_rescates    float64
n_aportes     float64
id             object
dtype: object
n_rescates    float64
n_aportes     float64
id              int64
dtype: object


,n_rescates,n_aportes,id
0,0.0,0.0,0
1,7.0,11.0,1


In [13]:
exp_lib.columns_cast(
    cuenta_banco,
    ["id"],
    "int64")
cuenta_banco.head(2)

id                        int64
clasificacion_cliente    object
dtype: object


,id,clasificacion_cliente
0,37146,segmento_1
1,37147,segmento_1


In [14]:
demograficos["fec_nacim"]=demograficos["fec_nacim"].apply(exp_lib.convert_to_date)
print(
    f"""
    Formato fecha a periodo
    {''.join(str(demograficos["fec_nacim"].to_list()[0]).split('-')[:2])}
    """
    )
exp_lib.columns_cast(
    demograficos,
    ["id"],
    "int64")
demograficos.head(2)


    Formato fecha a periodo
    194304
    
id                                int64
fec_nacim                datetime64[ns]
flg_empleado                     object
gls_nacioncorta                  object
gls_residencialarga              object
gls_profesionlarga               object
cod_sexo                         object
gls_cod_estcivillarga            object
gls_niv_educaclarga              object
gls_sep_bieneslarga              object
dtype: object


,id,fec_nacim,flg_empleado,gls_nacioncorta,gls_residencialarga,gls_profesionlarga,cod_sexo,gls_cod_estcivillarga,gls_niv_educaclarga,gls_sep_bieneslarga
0,10170,1943-04-23,N,Chile,Residencia Propia,PERIODISTA,F,Casado,Universitario Completo,SIN SEPARACIÓN
1,24533,1957-01-18,N,Chile,,OTRA,F,Casado,Medio,SIN SEPARACIÓN


In [15]:
deudas.columns

Index(['id', 'periodo', 'entidades', 'disponible', 'deuda_1', 'deuda_2',
       'deuda_3', 'deuda_4', 'deuda_5', 'mora_1', 'mora_2', 'mora_3', 'mora_4',
       'mora_5', 'mora_6', 'mora_7'],
      dtype='object')

In [16]:
list_name_file_1
# deudas
exp_lib.columns_cast(
    deudas,
    ['disponible', 'deuda_1', 'deuda_2',
       'deuda_3', 'deuda_4', 'deuda_5', 'mora_1', 'mora_2', 'mora_3', 'mora_4',
       'mora_5', 'mora_6', 'mora_7',],
    "float64")
exp_lib.columns_cast(
    deudas,
    ['id', 'periodo','entidades'],
    "int64")

id             object
periodo        object
entidades      object
disponible    float64
deuda_1       float64
deuda_2       float64
deuda_3       float64
deuda_4       float64
deuda_5       float64
mora_1        float64
mora_2        float64
mora_3        float64
mora_4        float64
mora_5        float64
mora_6        float64
mora_7        float64
dtype: object
id              int64
periodo         int64
entidades       int64
disponible    float64
deuda_1       float64
deuda_2       float64
deuda_3       float64
deuda_4       float64
deuda_5       float64
mora_1        float64
mora_2        float64
mora_3        float64
mora_4        float64
mora_5        float64
mora_6        float64
mora_7        float64
dtype: object


In [17]:
# list_name_file_1
exp_lib.columns_cast(
    id_label,
    ['id', 'label'],
    "int64")

id       int64
label    int64
dtype: object


In [23]:
df_cierre_patrimonio["fecha_cierre"]=\
    df_cierre_patrimonio["fecha_cierre"].apply(exp_lib.convert_to_date)
exp_lib.columns_cast(
    df_cierre_patrimonio,
    ["patrimonio"],
    "float64")
exp_lib.columns_cast(
    df_cierre_patrimonio,
    ["id"],
    "int64")
df_cierre_patrimonio.head(2)

patrimonio             float64
fecha_cierre    datetime64[ns]
id                      object
dtype: object
patrimonio             float64
fecha_cierre    datetime64[ns]
id                       int64
dtype: object


,patrimonio,fecha_cierre,id
0,11.804569,2022-02-28,21636
1,3.668090,2022-02-28,707


# Descripción de los datos

dataframes con información: 
- **Estatica** : 
  - id_label
  - demograficos
  - aportes_rescates_12m
  - cuenta_banco
- **Cambiante en el tiempo** :
  - df_cierre_patrimonio 
  - deudas

## Dataframes Estaticos

### Id_label.csv:
Base con id’s (columna id) de clientes junto con su etiqueta
(columna label) de 1 o 0 según corresponda. La columna label contiene 1’s
cuando un cliente retira parte de sus fondos (no importa cuanto, basta que
retire 1 peso para considerarse un 1) durante el mes siguiente (marzo). Por otro lado, la etiqueta de 0’s quiere decir que el cliente mantiene o aumenta
sus fondos en INVIERTE. (no importa cuanto, basta que agregue 1 peso
para considerarse un 0)



In [51]:
df_g = id_label.groupby("label").count().rename(
    columns={"id":"cantidad de cuentas"})
df_g["porcetaje"]=(df_g["cantidad de cuentas"]/ id_label.shape[0])
df_g

,cantidad de cuentas,porcetaje
label,,
0,31287,0.842271
1,5859,0.157729


### demograficos.csv: 

Datos demográficos de los clientes de INVIERTE.
1. fec_nacim:   
    > Fecha de nacimiento.
2. flg_empleado:    
    > S: El cliente es colaborador de YO INVIERTO. N: No es colaborador.
3. gls_nacioncorta:     
    > País de residencia del cliente.
4. gls_residencialarga:     
    > Categoría de la residencia donde vive el cliente.
5. gls_profesionlarga:  
    > Profesión del cliente.
6. gls_sep_bieneslarga:     
    > Indica si el cliente está casado o no. En caso de ser
vacío, el cliente está casado, si el valor es “sin separación” está separado
sin separación de bienes y “con separación” está separado con separación
de bienes.

In [97]:
demograficos.columns

Index(['id', 'fec_nacim', 'flg_empleado', 'gls_nacioncorta',
       'gls_residencialarga', 'gls_profesionlarga', 'cod_sexo',
       'gls_cod_estcivillarga', 'gls_niv_educaclarga', 'gls_sep_bieneslarga'],
      dtype='object')

In [52]:
demograficos.describe(include="all").T

<ipython-input-52-d25b4094788a>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
id,36970.0,NaN,NaN,NaN,NaT,NaT,18563.040438,10722.911939,0.0,9281.25,18562.5,27849.75,37145.0
fec_nacim,36941,19719,1973-11-30 00:00:00,12,1902-08-21,2023-07-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
flg_empleado,36970,2,N,36230,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gls_nacioncorta,36970,59,Chile,35763,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gls_residencialarga,36970,6,,17009,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gls_profesionlarga,36970,144,OTRA,15117,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cod_sexo,36970,4,M,20460,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gls_cod_estcivillarga,36970,8,Casado,13902,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gls_niv_educaclarga,36970,8,Universitario Completo,15870,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gls_sep_bieneslarga,36970,6,,23716,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:

df_merge = \
    pd.merge(id_label,demograficos,how="left", on= "id")
print(
f"""
Total de registros de id_label {id_label.shape}
Total de registros de demograficos {demograficos.shape}
Cuentas que poseen información demografica:
    {df_merge[df_merge.cod_sexo.notnull()].shape}
Cuentas sin información demografica:
    {df_merge[df_merge.cod_sexo.notnull() == False].shape}
""")



Total de registros de id_label (37146, 2)
Total de registros de demograficos (36970, 10)
cuentas que poseen información demografica:
    (36970, 11)
cuentas sin información demografica:
    (176, 11)



### cuenta_banco.csv:
Cuenta banco es un producto de YO INVIERTO
que te permite tener una tarjeta llamada inversionista_corriente. Si
tienes tarjeta en cuenta banco, los ejecutivos de cuenta banco te asignan
una clasificación segun tu perfil (variable o columna clasificacion_cliente).
4

In [91]:
df_g = cuenta_banco.groupby("clasificacion_cliente").count().rename(
    columns={"id":"cantidad de cuentas"})
df_g["porcetaje del total"]=(df_g["cantidad de cuentas"]/ cuenta_banco.shape[0]
                   ).apply(lambda x: f"{str(x*100)[:5]} %")
df_g

,cantidad de cuentas,porcetaje del total
clasificacion_cliente,,
segmento_1,16328,22.78 %
segmento_2,27788,38.77 %
segmento_3,24895,34.73 %
segmento_4,2653,3.701 %


In [86]:

df_merge = \
    pd.merge(id_label,cuenta_banco,how="left", on= "id")
print(
f"""
Total de registros de id_label {id_label.shape}
Total de registros de cuenta_banco {cuenta_banco.shape}
Cuentas que poseen información cuenta_banco:
    {df_merge[df_merge.clasificacion_cliente.notnull()].shape}
Cuentas sin información cuenta_banco:
    {df_merge[df_merge.clasificacion_cliente.notnull() == False].shape}
""")



Total de registros de id_label (37146, 2)
Total de registros de cuenta_banco (71664, 2)
Cuentas que poseen información cuenta_banco:
    (17331, 3)
Cuentas sin información cuenta_banco:
    (19815, 3)



In [114]:
17331/37146

0.4665643676304313

In [93]:
df_merge = \
    df_merge[df_merge.clasificacion_cliente.notnull()]
df_g = df_merge.groupby(
    ["clasificacion_cliente", "label"
     ]).count().rename(
    columns={"id":"cantidad de cuentas"})
df_g["porcetaje del total"]=(df_g["cantidad de cuentas"]/ df_merge.shape[0]
                   ).apply(lambda x: f"{str(x*100)[:5]} %")
df_g

cantidad de cuentas porcetaje del total
clasificacion_cliente label                                         
segmento_1            0                     1400             8.078 %
                      1                      290             1.673 %
segmento_2            0                     5022             28.97 %
                      1                     1016             5.862 %
segmento_3            0                     6721             38.78 %
                      1                     1681             9.699 %
segmento_4            0                      878             5.066 %
                      1                      323             1.863 %

### aportes_rescates_12m.csv:
Este archivo contiene un consolidado de
la suma de todos los aportes y rescates de los clientes durante 12 meses
(suma). Digamos, n_aportes = 14 quiere decir que el cliente x ha hecho
14 aportes en los últimos 12 meses. Por otro lado, n_rescates = 14 quiere
decir que el cliente ha rescatado 14 veces en los últimos 12 meses.

In [95]:
aportes_rescates_12m.head()

,n_rescates,n_aportes,id
0,0.0,0.0,0
1,7.0,11.0,1
2,2.0,15.0,2
3,10.0,37.0,3
4,4.0,4.0,4


In [96]:
aportes_rescates_12m.describe(include="all")

,n_rescates,n_aportes,id
count,37146.000000,37146.000000,37146.000000
mean,4.505142,9.932994,18572.500000
std,24.548381,32.295558,10723.270886
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,9286.250000
50%,0.000000,1.000000,18572.500000
75%,2.000000,6.000000,27858.750000
max,733.000000,882.000000,37145.000000


In [113]:

df_merge = \
    pd.merge(id_label,aportes_rescates_12m,how="left", on= "id")
print(
f"""
Total de registros de id_label {id_label.shape}
Total de registros de cuenta_banco {aportes_rescates_12m.shape}
Cuentas que poseen información cuenta_banco:
    {df_merge[df_merge.n_rescates.notnull()].shape}
Cuentas sin información cuenta_banco:
    {df_merge[df_merge.n_rescates.notnull() == False].shape}
""")



Total de registros de id_label (37146, 2)
Total de registros de cuenta_banco (37146, 3)
Cuentas que poseen información cuenta_banco:
    (37146, 4)
Cuentas sin información cuenta_banco:
    (0, 4)



## Dataframes Dinamicos

### Base carpeta cierre_patrimonio: df_cierre_patrimonio
Esta carpeta contiene archivos .csv
que tienen los cierres de patrimonios de los clientes al final del mes. Dig-
amos, en el archivo 1, el cliente 1 , en la fecha indicada por el archivo o
la columna fecha_cierre indica que el patrimonio al final de ese mes es el
número indicado en la columna patrimonio.

In [100]:
df_cierre_patrimonio.describe(include="all")

<ipython-input-100-ed67fefa119f>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.


,patrimonio,fecha_cierre,id
count,440542.000000,440584,440584.000000
unique,NaN,13,NaN
top,NaN,2022-12-31 00:00:00,NaN
freq,NaN,37027,NaN
first,NaN,2022-02-28 00:00:00,NaN
last,NaN,2023-02-28 00:00:00,NaN
mean,7.575020,NaN,18481.963165
std,3.519575,NaN,10721.608405
min,-1.703749,NaN,0.000000
25%,5.198911,NaN,9279.000000


In [111]:
df_last_patrimonio=df_cierre_patrimonio.groupby("id").max().reset_index()[["id","fecha_cierre"]]
df_last_patrimonio =\
      pd.merge(
    df_last_patrimonio,
    df_cierre_patrimonio,
    on=["id","fecha_cierre"],how="left")
df_last_patrimonio.describe(include="all")

<ipython-input-111-94ffa19f418b>:7: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.


,id,fecha_cierre,patrimonio
count,37139.000000,37139,37135.000000
unique,NaN,9,NaN
top,NaN,2023-02-28 00:00:00,NaN
freq,NaN,35956,NaN
first,NaN,2022-02-28 00:00:00,NaN
last,NaN,2023-02-28 00:00:00,NaN
mean,18571.311344,NaN,7.373061
std,10723.624240,NaN,3.584208
min,0.000000,NaN,-1.194022
25%,9284.500000,NaN,4.747993


In [112]:

df_merge = \
    pd.merge(id_label,df_last_patrimonio,how="left", on= "id")
print(
f"""
Total de registros de id_label {id_label.shape}
Total de registros de cuenta_banco {df_last_patrimonio.shape}
Cuentas que poseen información cuenta_banco:
    {df_merge[df_merge.fecha_cierre.notnull()].shape}
Cuentas sin información cuenta_banco:
    {df_merge[df_merge.fecha_cierre.notnull() == False].shape}
""")



Total de registros de id_label (37146, 2)
Total de registros de cuenta_banco (37139, 3)
Cuentas que poseen información cuenta_banco:
    (37139, 4)
Cuentas sin información cuenta_banco:
    (7, 4)



### deudas.csv: 
Resumen de las deudas por periodo que mantienen los clientes de INVIERTE con entidades crediticias.
1. **periodo**: 
    > Periodo en año y mes (YYYYMM) de la deuda.
2. **entidades**: 
    > Cantidad de entidades crediticias con las que el cliente mantiene créditos consumo.
3. **disponible**: 
    > Monto que el cliente tiene disponible para utilizar como deuda/prestamo.
4. **deuda_1 a deuda_4:** 
    > Monto adeudado tipo 1 a 4 (representan deudas que son independientes entre si).
5. **mora_1 a mora_4:** 
    > Monto adeudado impago en etapa 1 a 4 (a mayor etapa, mayor tiempo impago).
6. **mora_5 a mora_6:** 
    > Monto adeudado impago indirecto en etapa 1 y 2 (representan deudas que no fueron tomadas directamente por el cliente).
7. **mora_7** :     
    > Otros tipos de montos adeudado impagos.


In [116]:
deudas.describe(include="all")

,id,periodo,entidades,disponible,deuda_1,deuda_2,deuda_3,deuda_4,deuda_5,mora_1,mora_2,mora_3,mora_4,mora_5,mora_6,mora_7
count,367543.000000,367543.000000,367543.000000,367543.000000,367543.000000,367543.000000,367543.000000,367543.000000,367543.000000,367543.000000,367543.000000,367543.000000,367543.000000,367543.000000,367543.000000,367543.000000
mean,18538.994909,202223.221125,0.125604,8.702045,3.443526,6.770710,1.118041,0.017012,0.014763,0.117433,0.057008,0.124974,0.063228,0.043087,0.011472,0.002251
std,10724.894817,35.164300,0.376133,2.692261,5.334411,3.186154,3.266514,0.450368,0.394624,0.806171,0.594816,0.959526,0.693559,0.633651,0.349808,0.151064
min,0.000000,202203.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9215.000000,202206.000000,0.000000,7.901007,0.000000,5.505332,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,18546.000000,202209.000000,0.000000,9.367430,0.000000,7.543273,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,27827.000000,202212.000000,0.000000,10.439425,10.661544,8.988446,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,37145.000000,202302.000000,4.000000,15.556579,14.934640,14.496069,16.264216,14.397726,14.106924,12.491090,12.524875,12.875608,13.662888,14.973786,16.043781,12.427110


## Relación entre id_label y columnas atributos a disposición

### Filtros: 
1. `df_cierre_patrimonio` no tiene información de 7 clientes, estos clientes se deben eliminar.
2. cuentas sin información demografica: 176 Serán descartadas al cruzar con la tabla `demograficos`
3. `cuenta_banco ` solo posee registros para el 46,7% de las cuentas en `id_labe`, nueva columnas `sin_segmento`
4. 


### Columnas de interes a revisar:
```python
{
    "demograficos": ['fec_nacim', 'flg_empleado', 'gls_nacioncorta',
       'gls_residencialarga', 'gls_profesionlarga', 'cod_sexo',
       'gls_cod_estcivillarga', 'gls_niv_educaclarga', 'gls_sep_bieneslarga']
    "clasificacion_cliente":['clasificacion_cliente'],
    "aportes_rescates_12m" :['n_rescates','n_aportes'],
}
```